# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-30-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-11-events.csv', '/home/workspace/event_data/2018-11-14-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-28-events.csv', '/home/workspace/event_data/2018-11-25-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-12-events.csv', '/home/workspace/event_data/2018-11-27-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# TO-DO: Create a Keyspace 
keyspace_name = "eventdata"
try:
    session.execute("""
            CREATE KEYSPACE IF NOT EXISTS eventdata
            WITH REPLICATION =
            { 'class': 'SimpleStrategy', 'replication_factor': 1 }"""
        )
except Exception as e:
    print("Error creating KEYSPACE %s " % e)

#### Set Keyspace

In [7]:
# Connect KEYSPACE to the keyspace specified above
try:
    session.set_keyspace("eventdata")
except Exception as e:
    print("Error setting KEYSPACE %s " % e)

SyntaxError: invalid syntax (<ipython-input-7-9d90650d9c29>, line 1)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## Query 1

Find all rows given the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4.

For queries to be executed quickly and efficiently we need to define a table that would allow for execution of the following CQL query:
```
SELECT artist, song title and song's length FROM eventdata WHERE sessionID = 338 AND itemInSession = 4 
```

Let us now define a table with name *music_library*
with *Primary key* set to sessionId, and *Clustering key* itemInSession so that we can efficiently filter by this attributes later on.

In [8]:
query = """CREATE TABLE IF NOT EXISTS music_library (
                sessionId int,
                itemInSession int,
                length double,
                artist text,
                song text,
                userId int,
                PRIMARY KEY(sessionId, itemInSession))"""
try:
    rows = session.execute(query)
except Exception as e:
    print("Exception occcured on SELECT statement %s" % e)             

In [9]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    for line in csvreader:
        # Assign the INSERT statements into the `query` variable
        query = "INSERT INTO music_library (sessionId, itemInSession, length, artist, song, userId)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        # let's check that passed line is correct
        if len(line) == 11:
            artist, fName, gender, itemInSession, lName, length, level, location, sessionId, song, userId = line
            t = (int(sessionId), int(itemInSession), float(length), artist, song, int(userId))
            #print(t)
            session.execute(query,t)
        else:
            print(line)
            raise TypeError("not all arguments were provided for string formatting")

#### SELECT statement to verify the data was entered into the table

In [10]:

query = "SELECT artist, song, length FROM music_library WHERE sessionid = 338 AND itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print("Exception occcured on SELECT statement %s " % e)
    
for row in rows:
    print(row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.3073


## Query 2:
Find rows given only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182.

For queries to be executed quickly and efficiently we need to define a table that would allow for execution of the following CQL query:
```
SELECT itemInSession, artist, song, firstName, lastName FROM played_by_user WHERE userid = 10 AND sessionId = 182
```

Let us now define a table with name *played_by_user*
with composite *Primary key* of (sessionId, userId) and *Clustering key* itemInSession so that results can be ordered by this field.

In [12]:
query = """CREATE TABLE IF NOT EXISTS played_by_user (
                firstName text,
                lastName text,
                sessionId int,
                itemInSession int,
                artist text,
                song text,
                userId int,
                PRIMARY KEY((sessionId, userId), itemInSession))"""
try:
    rows = session.execute(query)
except Exception as e:
    print("Exception occcured on SELECT statement %s" % e)             

In [16]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    for line in csvreader:
        # Assign the INSERT statements into the `query` variable
        query = "INSERT INTO played_by_user (firstName, lastName, sessionId, itemInSession, artist, song, userId)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        # Assign which column element should be assigned for each column in the INSERT statement.
        # For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        if len(line) == 11:
            artist, firstName, gender, itemInSession, lastName, length, level, location, sessionId, song, userId = line
            t = (fName, lName, int(sessionId), int(itemInSession), artist, song, int(userId))
            session.execute(query,t)
        else:
            print(line)
            raise TypeError("not all arguments were provided for string formatting")

In [17]:
query = "SELECT itemInSession, artist, song, firstName, lastName FROM played_by_user WHERE userid = 10 AND sessionId = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print("Exception occcured on SELECT statement %s " % e)
    
for row in rows:
    print(row.iteminsession, row.artist, row.song, row.firstname, row.lastname)

0 Down To The Bone Keep On Keepin' On Isaac Valdez
1 Three Drives Greece 2000 Isaac Valdez
2 Sebastien Tellier Kilometer Isaac Valdez
3 Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Isaac Valdez


## Query 3
Find rows for every user name (first and last) in music app history who listened to the song 'All Hands Against His Own'


For queries to be executed quickly and efficiently let's define a table based on the following CQL query:
```
SELECT firstName, lastName FROM songs WHERE song='All Hands Against His Own'
```
Table name would be *songs*, with *Partition key* song and *Clustering key* userId to identify rows uniquely.

In [18]:
query = """CREATE TABLE IF NOT EXISTS songs (
                firstName text,
                lastName text,
                sessionId int,
                gender text,
                itemInSession int,
                length double,
                level text,
                artist text,
                song text,
                userId int,
                locationName text,
                PRIMARY KEY(song, userId))"""
try:
    rows = session.execute(query)
except Exception as e:
    print("Exception occcured on SELECT statement %s" % e)             

                    

In [19]:

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    for line in csvreader:
        # Assign the INSERT statements into the `query` variable
        query = "INSERT INTO songs (firstName, lastName, song, userId)"
        query = query + " VALUES (%s, %s, %s, %s)"
        # Assign which column element should be assigned for each column in the INSERT statement.
        if len(line) == 11:
            artist, fName, gender, itemInSession, lName, length, level, location, sessionId, song, userId = line
            t = (fName, lName, song, int(userId))
            session.execute(query,t)
        else:
            print(line)
            raise TypeError("not all arguments were provided for string formatting")


In [20]:
query = "SELECT firstName, lastName FROM songs WHERE song='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print("Exception occcured on SELECT statement %s " % e)
    
for row in rows:
    print(row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

Let's execute a function that drops the table by the provided name

In [21]:
def drop_table(name):
    query = "DROP TABLE IF EXISTS %s" % name
    try:
        rows = session.execute(query)
    except Exception as e:
        print("Exception occcured on DROP statement " % e)

drop_table("music_library")
drop_table("played_by_user")
drop_table("songs")

### Close the session and cluster connection¶

In [22]:
session.shutdown()
cluster.shutdown()